## An Exploration of Chicago's Array of Things API (via python client)
Array of Things has expanded in Chicago to include more than 100 continuous sensors across the city.

**Resources**
* [Home Page](https://arrayofthings.github.io)
* [python client](https://github.com/UrbanCCD-UChicago/aot-client-py)
* [API documentation](https://arrayofthings.docs.apiary.io)

**TODOs**
  * group measurements by larger cluster than sensor? by project?
  * efficiently query regular interval of data, to annimate
  * normalize measurements, choose reasonable circle radii and color
    * map different sensors to different visualizations

In [ ]:
 # !pip install aot-client

In [ ]:
from aot_client import AotClient

client = AotClient()

# What are the methods/properties of the client?
[_ for _ in dir(client) if not _.startswith('_')]

It appears results will be paginated and we can also pass a filter. From the docs website I see that users may pass a timestamp filter in ISO 8601 format. Let's get measurements from the past 15 minutes.

In [ ]:
import datetime
t = (datetime.datetime.now() - 
     datetime.timedelta(days=7))
t = t.strftime(r'%m/%d/%Y')
t

In [ ]:
import datetime

import pandas as pd
from aot_client import AotClient
from aot_client import F


def time_x_mins_ago(minutes:int):
    '''Get formatted time to pass to API filter, relative to current time
    '''
    t = (datetime.datetime.now() - 
         datetime.timedelta(minutes=minutes) + 
         datetime.timedelta(hours=5))  # convert timezone from central to UTC
    t = t.isoformat()
    
    return t[0:19]


def unpack_response(response, page_limit=1000):
    try:
        pages = []
        for i, page in enumerate(response):
            if i + 1 > page_limit:
                break
            pages.extend(page.data)
    except HTTPError as e:
        print(e)    
    finally:
        return pages


def process_observations(obs_df):
    obs_df = obs_df.copy()
    obs_df['timestamp'] = pd.to_datetime(obs_df['timestamp'], utc=True)
    obs_df['timestamp'] = obs_df['timestamp'].dt.tz_convert('US/Central')
    
    # extract lat/lon to columns
    obs_df['coords'] = obs_df['location'].apply(
        lambda x: x['geometry']['coordinates'])
    obs_df[['lon', 'lat']] = pd.DataFrame(
        obs_df['coords'].tolist(), columns=['lon', 'lat'])
    obs_df = obs_df.drop(columns=['coords'])
    
    # fix positive lon values
    mask = obs_df['lon'] > 0
    if sum(mask) > 0:
        print(f'fixed {sum(mask)} rows with positive lon value')
        obs_df.loc[mask, 'lon'] = obs_df.loc[mask, 'lon'] * -1

    # remove lat/lon values at 0
    mask = (obs_df['lon'] != 0) & (obs_df['lat'] != 0)
    if len(obs_df) - sum(mask) > 0:
        print(f'removed {len(obs_df) - sum(mask)} rows with lat/lon at 0')
        obs_df = obs_df.loc[mask]

    # remove lat values less than 40 degrees
    mask = (obs_df['lat'] > 40)
    if len(obs_df) - sum(mask) > 0:
        print(f'removed {len(obs_df) - sum(mask)} '
              'rows with lat/lon outside Chicago region')
        obs_df = obs_df.loc[mask]
    
    return obs_df

In [ ]:
client = AotClient()

# create filter
f = F('size', '90000')
f &= ('timestamp', 'ge', time_x_mins_ago(5))
# f &= ('sensor', 'image.image_detector.person_total')
# f &= ('time_bucket', 'avg:1 hour')
# f &= ('sensor', 'image.image_detector.car_total')
# f &= ('sensor', 'metsense.tsys01.temperature')

response = client.list_observations(filters=f)
print(response.current_link)
pages = unpack_response(response, page_limit=1)
print(len(pages))
obs_df = pd.DataFrame(pages)
# obs_df = process_observations(obs_df)
print(f"shape: {obs_df.shape}")
obs_df.head()

In [ ]:
len(obs_df['node_vsn'].unique())

In [ ]:
obs_df.groupby('node_vsn')['sensor_path'].nunique()

In [ ]:
import folium
import folium.plugins

def map(df):
    m = folium.Map(location=[df['lat'].mean(), 
                             df['lon'].mean()],
                   tiles='CartoDB dark_matter',
                   zoom_start=10)

    for i, r in df.iterrows():
        folium.CircleMarker(
            location=(r['lat'], r['lon']),              
#             radius=3,
#             color=r['color'],
#             weight=0.5,
            tooltip=f"{r['timestamp']}<br>{r['value']} {r['uom']}",
#             popup=folium.Popup(f"{r['value']} {r['uom']}", max_width=500),
            fill=True
        ).add_to(m)

    folium.plugins.Fullscreen(
        position='topright',
        force_separate_button=True
    ).add_to(m)

    return m

In [ ]:
response.current_link

In [ ]:
len(sensors_df['path'].unique())

In [ ]:
obs_df.head()

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

sensors = client.list_sensors()
sensors_df = pd.DataFrame(sensors.data)
sensors_df.head()

@interact_manual
def choose_sensor(sensor=sensors_df['path'].unique()):
    client = AotClient()
    f = F('sensor', sensor)

    response = client.list_observations(filters=f)
    print('API call:', response.current_link)
    pages = unpack_response(response, page_limit=5)
    
    if not pages:
        print('No data found.')
        return None
    
    obs_df = pd.DataFrame(pages)
    obs_df = process_observations(obs_df)
    print(obs_df.shape)
    
    return map(obs_df.drop_duplicates(['lon', 'lat']))

### Plotly

In [ ]:
# initial configuration
import plotly
from os import getenv
from dotenv import load_dotenv

load_dotenv()

plotly.tools.set_credentials_file(
    username=getenv('PLOTLY_USER'), 
    api_key=getenv('PLOTLY_API_KEY')
)

In [ ]:
# exploring plots
import plotly.plotly as py
import plotly.graph_objs as go

trace0 = go.Scatter(
    x=[1, 2, 3, 4],
    y=[10, 15, 13, 17]
)
trace1 = go.Scatter(
    x=[1, 2, 3, 4],
    y=[16, 5, 11, 9]
)
data = [trace0, trace1]

plot_url = py.plot(data, filename = 'basic-line', auto_open=False)
print(plot_url)

In [ ]:
import pandas as pd

df = pd.read_csv('data/query_Environmental_Temperature.csv')

In [ ]:
from app.plotting import make_map, make_line_plot, make_hourly_bar_plot

print(make_map(df))
print(make_line_plot(df, 'Temperature'))
print(make_hourly_bar_plot(df, 'Temperature'))

In [ ]:
from app.aot_feed import *
from app.aot_archive import *

In [ ]:
df = query_aot('Ozone')

In [ ]:
df

In [ ]:
sensor_hrf='Temperature'
size_per_page=10000
page_limit=1
mins_ago=12*60

In [ ]:
sensors = (
    SENSOR_DF.loc[SENSOR_DF['sensor_measure']==sensor_hrf, 'sensor_path']
             .unique()
)

print(sensors)

In [ ]:
time_x_mins_ago(12*60)

In [ ]:
client = AotClient()

pages = []
for sensor in sensors:    
    f = F('size', str(100))
    f &= ('sensor', sensor)
    f &= ('timestamp', 'ge', time_x_mins_ago(mins_ago))
    response = client.list_observations(filters=f)
    pages.extend(unpack_response(response, page_limit=page_limit))

obs_df = pd.DataFrame(pages)
obs_df = process_observations(obs_df)
print(obs_df['node_vsn'].unique())

In [ ]:
obs_df[['timestamp', 'value']].plot(x='timestamp', y='value');

In [ ]:
obs_df[['timestamp', 'value']].plot

In [ ]:
# list observations
client = AotClient()

f = F()
# f &= ('sensor', 'metsense.tsys01.temperature')
# f &= ('sensor', 'wagman.wagman_cu.ep')

response = client.list_observations(filters=f)
pages = unpack_response(response, page_limit=1)
obs_df = pd.DataFrame(pages)
# obs_df = process_observations(obs_df)

In [ ]:
process_observations(obs_df)

In [ ]:
# list nodes
client = AotClient()
response = client.list_nodes(filters=F('size', '1000'))
pages = unpack_response(response)
df = pd.DataFrame(pages)

# extract lat/lon to columns
df['coords'] = df['location'].apply(
    lambda x: x['geometry']['coordinates'])
df[['lon', 'lat']] = pd.DataFrame(
    df['coords'].tolist(), columns=['lon', 'lat'])
df = df.drop(columns=['coords'])

df.head()

In [ ]:
# list sensors
client = AotClient()

response = client.list_sensors(filters=F('size', '1000'))
pages = unpack_response(response)
df = pd.DataFrame(pages)

df.head()

# Import data from AoT Archive

In [ ]:
import pandas as pd

from app.aot import (
    load_aot_archive_day, 
    clean_aot_archive_obs, 
    get_nodes, get_sensors
)

In [ ]:
date = '2019-05-23'
df = load_aot_archive_day(day=date)
df = clean_aot_archive_obs(df)

### Publish data to production database

In [ ]:
from os import getenv

from dotenv import load_dotenv
import pandas as pd
import sqlalchemy

load_dotenv()

#### Method 1 - too slow and blows up in RAM

In [ ]:
engine = sqlalchemy.create_engine(getenv("HEROKU_DB_URL"))

In [ ]:
engine.execute("SELECT * FROM observation").fetchone()

In [ ]:
engine.execute("SELECT * FROM node").fetchone()

In [ ]:
engine.execute("SELECT * FROM sensor").fetchone()

In [ ]:
# filter df to the sensors we care about
sensors = engine.execute("SELECT sensor.sensor_path FROM sensor").fetchall()
df = df.loc[df['sensor_path'].isin([t[0] for t in sensors])]

max_id = engine.execute("SELECT max(observation.id) FROM observation").fetchone()[0]

if not max_id:
    max_id = 0

df['id'] = list(range(max_id + 1, max_id + len(df) + 1))

In [ ]:
df.to_sql(
    'observation', con=engine, if_exists='append', 
    index=False, chunksize=1000
)

#### Method 2

In [ ]:
from urllib.parse import urlparse
import psycopg2
import os

def upload_df_to_db(df, table_name):
    result = urlparse(os.getenv("HEROKU_DB_URL"))
    username = result.username
    password = result.password
    database = result.path[1:]
    hostname = result.hostname
    conn = psycopg2.connect(
        database = database,
        user = username,
        password = password,
        host = hostname
    )
    
    cur = conn.cursor()

    # get column names and order
    cur.execute(f"SELECT * FROM {table_name}")
    cols = [desc[0] for desc in cur.description]

    csv_path = 'to_upload.csv'
    df[cols].to_csv(csv_path, index=False)

    with open(csv_path, 'r') as f:
        next(f) # Skip the header row.
        cur.copy_from(f, table_name, sep=',')
        conn.commit()
    
    os.remove(csv_path)
    
    cur.close()
    conn.close()